### Eliminating warning in jupyter notebook

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
import os
from keras.models import Sequential # Initialise our neural network model as a sequential network
from keras.layers import Conv2D # Convolution operation
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector.
from keras.layers import Dense # Perform the full connection of the neural network
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from PIL import Image
import cv2
import numpy as np
from sklearn.metrics import accuracy_score
from skimage import io, transform

Using TensorFlow backend.


### Convolutional Neural Network — Deep Learning in python

In [6]:
def cnn_classifier():
    cnn = Sequential()
    cnn.add(Conv2D(8, (3,3), input_shape = (50, 50, 3), padding='same', activation = 'relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    cnn.add(Conv2D(16, (3,3), padding='same', activation = 'relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    cnn.add(Flatten())
    cnn.add(Dense(128, activation = 'relu'))
    cnn.add(Dense(2, activation = 'softmax'))
    cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    print(cnn.summary())
    return cnn

def reshaped_image(image):
    return transform.resize(image,(50,50,3)) 

def load_images_from_folder():
    Images = os.listdir("./Dataset/")
    train_images = []
    train_labels = []
    for image in Images:
            if image[-4:] == 'jpeg':
                path = os.path.join("./Dataset/", image)
                img = cv2.imread(path)
                train_images.append(reshaped_image(img))
                label_file = image[:-5] + '.txt'
                with open("./Dataset"+"/"+label_file) as f:
                    content = f.readlines()
                    label = int(float(content[0]))
                    l = [0, 0]
                    l[label] = 1 # 1=car and 0=not car
                    train_labels.append(l)
    return np.array(train_images), np.array(train_labels)

def load_images_from_folder_for_test():
    Images = os.listdir("./Dataset/test1")
    test_images = []
    test_labels = []
    for image in Images:
            if image[-4:] == 'jpeg':
                path = os.path.join("./Dataset/test1", image)
                img = cv2.imread(path)
                test_images.append(reshaped_image(img))
                label_file = image[:-5] + '.txt'
                with open("./Dataset/test1"+"/"+label_file) as f:
                    content = f.readlines()
                    label = int(float(content[0]))
                    l = [0, 0]
                    l[label] = 1 # 1=car and 0=not car
                    test_labels.append(l)
    return np.array(test_images), np.array(test_labels)
        
def train_test_split(train_data, train_labels, fraction):
    index = int(len(train_data)*fraction)
    return train_data[:index], train_labels[:index], train_data[index:], train_labels[index:]

train_data, train_labels = load_images_from_folder()
test_data, test_labels = load_images_from_folder_for_test()
fraction = 0.8
#train_data, train_labels, test_data, test_labels = train_test_split(train_data, train_labels, fraction)
print ("Train data size: ", len(train_data))
print ("Test data size: ", len(test_data))

cnn = cnn_classifier()

print ("Train data shape: ", train_data.shape)
#print ("Test data shape: ", train_labels.shape)

idx = np.random.permutation(train_data.shape[0])
cnn.fit(train_data[idx], train_labels[idx], epochs = 10)
predicted_test_labels = np.argmax(cnn.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)

#print ("Actual test labels:", test_labels)
#print ("Predicted test labels:", predicted_test_labels)

print ("Accuracy score:", accuracy_score(test_labels, predicted_test_labels))


('Train data size: ', 1531)
('Test data size: ', 18)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 50, 50, 8)         224       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 25, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 25, 25, 16)        1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2704)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               346240    
_________________________________________________________________
dense_6 (Dense)        

In [4]:
print ("Actual test labels:", test_labels)
print ("Predicted test labels:", predicted_test_labels)



('Actual test labels:', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]))
('Predicted test labels:', array([0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]))
